# Lesson 1: Advanced RAG Pipeline

In [1]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [2]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [3]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'>
Doc ID: 073b9f46-a1b5-4a28-8857-f7a7aa1b0b21
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


## Basic RAG pipeline

In [4]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [5]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
query_engine = index.as_query_engine()

In [7]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
print(str(response))

Develop a side hustle, ensure the project will help you grow technically, collaborate with good teammates, and consider if the project can be a stepping stone to larger projects.


## Evaluation setup using TruLens

In [8]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [9]:
# You can try your own question:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)

In [10]:
print(eval_questions)

['What are the keys to building a career in AI?', 'How can teamwork contribute to success in AI?', 'What is the importance of networking in AI?', 'What are some good habits to develop for a successful career?', 'How can altruism be beneficial in building a career?', 'What is imposter syndrome and how does it relate to AI?', 'Who are some accomplished individuals who have experienced imposter syndrome?', 'What is the first step to becoming good at AI?', 'What are some common challenges in AI?', 'Is it normal to find parts of AI challenging?', 'What is the right AI job for me?']


In [11]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


For the classroom, we've written some of the code in helper functions inside a utils.py file.  
- You can view the utils.py file in the file directory by clicking on the "Jupyter" logo at the top of the notebook.
- In later lessons, you'll get to work directly with the code that's currently wrapped inside these helper functions, to give you more options to customize your RAG pipeline.

In [12]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [13]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

/usr/local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [14]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [15]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Answer Relevance_calls,Context Relevance_calls,Groundedness,Groundedness_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_b55b549b36095bc0f434bf3fe95d2495,"""What are the keys to building a career in AI?""","""Learning foundational technical skills, worki...",-,"{""record_id"": ""record_hash_b55b549b36095bc0f43...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-01-13T17:04:53.129317"", ""...",2025-01-13T17:04:54.628270,1.0,1.00,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'prompt': 'What are the keys to bui...,NaN,NaN,1,2108,0.003207
1,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_25810bf7b5c5cc8953c1f025fb758bc2,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by a...",-,"{""record_id"": ""record_hash_25810bf7b5c5cc8953c...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-01-13T17:04:54.753865"", ""...",2025-01-13T17:04:56.232145,0.9,0.50,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'prompt': 'How can teamwork contrib...,1.0,[{'args': {'source': 'Hopefully the previous c...,1,1694,0.002576
2,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_b80fcef49e313833e3dc3004216c89ed,"""What is the importance of networking in AI?""","""Networking is crucial in AI as it helps indiv...",-,"{""record_id"": ""record_hash_b80fcef49e313833e3d...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-01-13T17:04:56.345503"", ""...",2025-01-13T17:04:57.764916,0.9,0.75,[{'args': {'prompt': 'What is the importance o...,[{'args': {'prompt': 'What is the importance o...,NaN,NaN,1,1703,0.002593
3,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_ff3b7b9e815ee31b9d7974f3aeeb3dae,"""What are some good habits to develop for a su...","""Developing good habits in areas such as eatin...",-,"{""record_id"": ""record_hash_ff3b7b9e815ee31b9d7...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-01-13T17:04:57.884556"", ""...",2025-01-13T17:04:59.211101,1.0,0.95,[{'args': {'prompt': 'What are some good habit...,[{'args': {'prompt': 'What are some good habit...,NaN,NaN,1,1632,0.002466
4,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_0d37f8f4570885f006663d0e8957aea5,"""How can altruism be beneficial in building a ...","""Helping others during your career journey can...",-,"{""record_id"": ""record_hash_0d37f8f4570885f0066...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-01-13T17:04:59.321908"", ""...",2025-01-13T17:04:59.983932,1.0,NaN,[{'args': {'prompt': 'How can altruism be bene...,NaN,NaN,NaN,0,1610,0.002423


In [16]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at https://s172-29-11-8p38560.lab-aws-production.deeplearning.ai/ .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Advanced RAG pipeline

### 1. Sentence Window retrieval

In [17]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [18]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [19]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [20]:
window_response = sentence_window_engine.query(
    "how do I get started on a personal project in AI?"
)
print(str(window_response))

To get started on a personal project in AI, you can begin by identifying a project that aligns with your career goals and interests. It's important to choose a project that is responsible, ethical, and beneficial to people. Once you have selected a project, you can follow the steps outlined in the chapters provided, such as scoping the project, executing it with an eye toward career development, and building a portfolio that demonstrates skill progression. By following these guidelines, you can embark on a personal AI project that not only enhances your skills but also makes a positive impact in the field.


In [21]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [22]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

What are the keys to building a career in AI?
Learning foundational technical skills, working on projects, finding a job, and being part of a supportive community are the keys to building a career in AI.
How can teamwork contribute to success in AI?
Teamwork can contribute to success in AI by allowing individuals to leverage the diverse skills and perspectives of their colleagues. Collaborating with others can lead to more innovative solutions, effective problem-solving, and the ability to tackle larger projects that require a combination of technical expertise and leadership skills. Additionally, working in a team can create a supportive environment where team members can learn from each other, share insights, and collectively drive projects forward towards successful outcomes.
What is the importance of networking in AI?
Networking in AI is crucial as it can provide valuable insights, guidance, and opportunities for individuals looking to advance in the field. By connecting with profe

In [23]:
tru.get_leaderboard(app_ids=[])

,Answer Relevance,latency,total_cost
app_id,,,
Sentence Window Query Engine,1.0,1.363636,0.000823


In [24]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-29-11-8p38560.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### 2. Auto-merging retrieval

In [25]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

In [26]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [27]:
auto_merging_response = automerging_query_engine.query(
    "How do I build a portfolio of AI projects?"
)
print(str(auto_merging_response))

> Merging 1 nodes into parent node.
> Parent node id: a024a885-b054-4b50-bcf7-1d0723ed30eb.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

> Merging 1 nodes into parent node.
> Parent node id: 885e320d-9c09-49b1-ad02-dfec7b0a639f.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

Building a portfolio of AI projects involves showcasing a progression from simple to complex undertakings over time. It is important to be able to effectively communicate your thinking to demonstrate the value of your work and gain the trust of others. Identifying ideas that are worth working on is a crucial skill for an AI architect, and gaining experience by working on projects across various industries can help in building a diverse portfolio.


In [28]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [29]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f9e280d3ac0 is calling an instrumented method <function BaseQueryEngine.query at 0x7f9f9e68b6d0>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f9e7c264250) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f9e280d3ac0 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f9f996feb00>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f9e7c264250) using this function.
A new object of type <class 'llama_index.retrievers.auto_merging_retriever.AutoMergingRetriever'> at 0x7f9e680ecb80 is calling an instrumented method <function BaseRetriever.retrieve at 0x7f9f9e68aa70>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other object (0x7

> Merging 2 nodes into parent node.
> Parent node id: fff7f794-2d0c-4494-8f95-84216e57dad3.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: 1cd286fe-9cbd-4275-83d9-e42602562261.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f9e280d3a30 is calling an instrumented method <function CompactAndRefine.get_response at 0x7f9f9dc8f5b0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f9e7c266bf0) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f9e280d3a30 is calling an instrumented method <function Refine.get_response at 0x7f9f9d0c7370>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f9e7c266bf0) using this function.
A new object of type <class 'llama_index.llm_predictor.base.LLMPredictor'> at 0x7f9f991a51c0 is calling an instrumented method <function LLMPredictor.predict at 0x7f9faab691b0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthes

What are the keys to building a career in AI?
Learning foundational technical skills, working on projects, finding a job, and being part of a community are essential keys to building a career in AI. Additionally, collaborating with others, influencing, and being influenced by team members are critical aspects for success in AI.


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f9e280d3a30 is calling an instrumented method <function Refine.get_response at 0x7f9f9d0c7370>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f9e7c266bf0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f9e280d3ac0 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f9f996feb00>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f9e7c264250) using this function.


...

What are some common challenges in AI?
Some common challenges in AI include the highly iterative nature of AI projects, difficulties in project management due to uncertainties in time estimates for achieving target accuracy, and technical challenges that researchers and professionals in the field often face.
> Merging 3 nodes into parent node.
> Parent node id: 384762dc-4a00-49ae-b766-4a26126a7655.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...

> Merging 1 nodes into parent node.
> Parent node id: 104391f9-9d1d-4cb5-a81b-0858730930f8.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f9e280d3a30 is calling an instrumented method <function Refine.get_response at 0x7f9f9d0c7370>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f9e7c266bf0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f9e280d3ac0 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f9f996feb00>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f9e7c264250) using this function.


Is it normal to find parts of AI challenging?
It is normal to find parts of AI challenging.
> Merging 1 nodes into parent node.
> Parent node id: ab320d02-97aa-4058-87c0-c8e76eecf77b.
> Parent node text: PAGE 31Finding the Right 
AI Job for YouCHAPTER 9
JOBS

> Merging 1 nodes into parent node.
> Parent node id: aa932a15-b64d-4140-bd23-d827eff128af.
> Parent node text: If you’re leaving 
a job, exit gracefully. Give your employer ample notice, give your full effort...

> Merging 1 nodes into parent node.
> Parent node id: 3c58bf86-934c-4e7f-80f0-072406ee948e.
> Parent node text: PAGE 28Using Informational 
Interviews to Find 
the Right JobCHAPTER 8
JOBS

> Merging 1 nodes into parent node.
> Parent node id: 0164ba11-b486-4a5c-8bfb-7d6d8545adc8.
> Parent node text: PAGE 31Finding the Right 
AI Job for YouCHAPTER 9
JOBS

> Merging 1 nodes into parent node.
> Parent node id: 434c5768-1b18-46c5-9aae-eea511d76d0e.
> Parent node text: PAGE 28Using Informational 
Interviews to Find 
the Right

A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f9e280d3a30 is calling an instrumented method <function Refine.get_response at 0x7f9f9d0c7370>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f9e7c266bf0) using this function.


What is the right AI job for me?
The right AI job for you would likely involve transitioning into a role or industry that aligns with your current expertise or experience. It may be easier to switch either roles or industries first before attempting to switch both simultaneously when looking for your first job in AI.


In [30]:
tru.get_leaderboard(app_ids=[])

,latency,total_cost
app_id,,
Automerging Query Engine,2.181818,0.00087


In [31]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-29-11-8p38560.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>